In [1]:
import numpy as np
np.random.seed(1234)

# Strategies

## auxiliar functions

In [2]:
def get_neighbors(index, clockwise):
    neighbors = []
    if clockwise:
        neighbors.append((index - 1) % 8)
        neighbors.append((index + 1) % 8)
    else:
        neighbors.append((index + 1) % 8)
        neighbors.append((index - 1) % 8)
    return neighbors

In [3]:
def is_corner(dir):
    switch = {
        0: True,
        1: False,
        2: True,
        3: False,
        4: True,
        5: False,
        6: True,
        7: False
    }
    return switch.get(dir, False)
    

In [4]:
if (is_corner(0)):
    print("Corner")
else:
    print("Not a corner")

Corner


## Random Movement

In [5]:
def random_movement():
    return np.random.randint(0, 8)
random_movement()

7

## Rabbit Strategies
* clockwise_escape()

In [6]:
# This function receives the current state (f) of 
# the system and returns the next state.
# The current state (f) is an array of eight integers, 
# f[i] := amount of foxes in the i-th direction
# state index, same as directions:
# 0 1 2
# 7   3
# 6 5 4

def clockwise_escape(f, preferred_direction, clockwise):
    neighbors = get_neighbors(preferred_direction, clockwise)
    if not ( f[preferred_direction] 
            or f[neighbors[0]] 
            or f[neighbors[1]] ):
        return preferred_direction # safe from visible foxes

    current_direction = preferred_direction
    evaluated_safer_directions = 1
    while evaluated_safer_directions < 8:
        if clockwise:
            current_direction = (current_direction + 1) % 8
        else:
            current_direction = (current_direction - 1) % 8

        neighbors = get_neighbors(current_direction, clockwise)
        if not ( f[current_direction] 
                or f[neighbors[0]] 
                or f[neighbors[1]] ):
            return current_direction # safe from visible foxes
        evaluated_safer_directions += 1

    if not (f[preferred_direction]):
        return preferred_direction

    evaluated_unsafe_directions = 1
    while evaluated_unsafe_directions < 8:
        if clockwise:
            current_direction = (current_direction + 1) % 8
        else:
            current_direction = (current_direction - 1) % 8

        if not (f[current_direction]):
            return current_direction
        evaluated_unsafe_directions += 1
    
    return np.argmin(f)
        
        

In [7]:
clockwise_escape([10, 0, 1, 1, 3, 1, 0, 2], 5, False)

1

## Fox Strategies
* chasing_1

In [8]:
# This function receives the current state (r) of 
# the system and returns the next state.
# The current state (r) is an array of eight integers, 
# r[i] := amount of rabbits in the i-th direction
# state index, same as directions:
# 0 1 2
# 7   3
# 6 5 4


def chasing_1(r, preferred_direction, clockwise, diagonal_prediction):
    if not( np.any(r) ):
        return preferred_direction
    best_direction = np.argmax(r)
    neighbors = get_neighbors(best_direction, clockwise)
    if is_corner( best_direction ) or not( diagonal_prediction ):
        return neighbors[1]
    else:
        return get_neighbors(neighbors[1], clockwise)[1]


In [9]:
# 0 1 2
# 7   3
# 6 5 4
r = [0, 0, 2, 0, 0, 0, 0, 1]

chasing_1(r, 0, True, False)

3

In [10]:
rabbit_amounts = [0, 0, 1, 3, 4, 1, 0, 0]
chasing_1(rabbit_amounts, 2, clockwise = True, diagonal_prediction = True)

5

In [11]:
np.argmax(rabbit_amounts)

4

# Grid

## valid_position()

In [12]:
def valid_position(row, col, shape):
    height, width, aux = shape
    if row == -1:
        row = height - 1
    elif row == height:
        row = 0

    if col == -1:
        col = width - 1
    elif col == width:
        col = 0
    
    return row, col

## get_animals_around()

In [13]:
# [row, column, [rabbits, foxes]]
height = 3
width = 4

grid = np.zeros((height, width, 2), dtype=int)

c = 0
for row in range(height):
    for col in range(width):
        grid[row, col, 0] = c
        grid[row, col, 1] = c + 1
        c += 2
grid

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[16, 17],
        [18, 19],
        [20, 21],
        [22, 23]]])

In [14]:
# [row, column, [rabbits, foxes]]

# 0 1 2
# 7   3
# 6 5 4

def get_animals_around(grid, row, col, animal):

    if animal == "rabbits":
        animal_index = 0
    elif animal == "foxes":
        animal_index = 1

    height, width = grid.shape[:2]
    if valid_position(row, col, grid.shape) != (row, col):
        print("Invalid position: ({}, {})".format(row, col))
        return None

    map = np.zeros(8, dtype=int)
    
    for i in range(3):
        valid_row, valid_col = valid_position(row - 1, col - 1 + i, grid.shape)
        map[i] = grid[valid_row, valid_col, animal_index]
      
    valid_row, valid_col = valid_position(row, col + 1, grid.shape)
    map[3] = grid[valid_row, valid_col, animal_index]    

    for i in range(3):
        valid_row, valid_col = valid_position(row + 1, col + 1 - i, grid.shape)
        map[4 + i] = grid[valid_row, valid_col, animal_index]    
    
    valid_row, valid_col = valid_position(row, col - 1, grid.shape)
    map[7] = grid[valid_row, valid_col, animal_index]

    return map

In [15]:
# height = 3
# width = 4

# 0 1 2
# 7   3
# 6 5 4

# 0  2  4  6
# 8  10 12 14
# 16 18 20 22


get_animals_around(grid, 2, 3, "rabbits")

array([12, 14,  8, 16,  0,  6,  4, 20])

# Evolution

## breed_rabbits()

In [16]:
# [row, column, [rabbits, foxes]]
height = 3
width = 4

grid = np.zeros((height, width, 2), dtype=int)

c = 0
for row in range(height):
    for col in range(width):
        grid[row, col, 0] = c
        grid[row, col, 1] = c + 1
        c += 2
grid

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[16, 17],
        [18, 19],
        [20, 21],
        [22, 23]]])

In [17]:
def breed_rabbits(map):
    animal_index = 0
    map[:,:,animal_index] += map[:,:,animal_index]

In [18]:
#grid[:,:,1] += grid[:,:,1]
breed_rabbits(grid)
grid

array([[[ 0,  1],
        [ 4,  3],
        [ 8,  5],
        [12,  7]],

       [[16,  9],
        [20, 11],
        [24, 13],
        [28, 15]],

       [[32, 17],
        [36, 19],
        [40, 21],
        [44, 23]]])

## foxes_hunt_and_breed()

In [19]:
# [row, column, [rabbits, foxes]]
width = 2
height = 3

grid = np.zeros((height, width, 2), dtype=int)

grid[0,0,0] = 1
grid[0,0,1] = 2

grid[0,1,0] = 3
grid[0,1,1] = 0

grid[1,0,0] = 2
grid[1,0,1] = 1

grid[1,1,0] = 0
grid[1,1,1] = 3

grid[2,0,0] = 3
grid[2,0,1] = 3

grid[2,1,0] = 3
grid[2,1,1] = 5

grid 

array([[[1, 2],
        [3, 0]],

       [[2, 1],
        [0, 3]],

       [[3, 3],
        [3, 5]]])

In [20]:
np.array(
    [[[1,2], [3,0]],
    [[2,1], [0,3]],
    [[3,3], [3,5]]] )

array([[[1, 2],
        [3, 0]],

       [[2, 1],
        [0, 3]],

       [[3, 3],
        [3, 5]]])

In [21]:
grid

array([[[1, 2],
        [3, 0]],

       [[2, 1],
        [0, 3]],

       [[3, 3],
        [3, 5]]])

In [22]:
def foxes_hunt_and_breed(map):
    for r in range(map.shape[0]):
        for c in range(map.shape[1]):
            if (map[r,c,:] == 0).any():
                continue
            else:
                hunted_rabbits = np.min( map[r,c,:] )
                map[r, c, 0] -= hunted_rabbits
                map[r, c, 1] += hunted_rabbits
    return map
    

In [23]:
foxes_hunt_and_breed(grid)

array([[[0, 3],
        [3, 0]],

       [[1, 2],
        [0, 3]],

       [[0, 6],
        [0, 8]]])

In [24]:
m1 = np.zeros(grid.shape[:2], dtype=int)
m1[0,1] = 1
m1[1,0] = 1
m1[2,0] = 2
m1[2,1] = 2

m1


array([[0, 1],
       [1, 0],
       [2, 2]])

## evolve()

In [25]:
# returns the state of the system after one time unit

def evolve(rabbits, foxes, rabbit_strategies, fox_strategies, edge_length):
    pass
    

# Graphics

In [26]:
import numpy as np
import pygame
# pygame.init()

pygame 2.1.3.dev8 (SDL 2.26.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [27]:
pygame.display.init()

In [28]:
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# For each cell:
WIDTH = 30
HEIGHT = 30

MARGIN = 5

# For the grid:
ROWS = 10
COLUMNS = 10

In [29]:
# [row, column, [rabbits, foxes]]

grid = np.zeros((ROWS, COLUMNS, 2), dtype=int)
grid

array([[[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 

In [30]:
pygame.init()

WINDOWS_SIZE = [500, 500]
screen = pygame.display.set_mode(WINDOWS_SIZE)
pygame.display.set_caption("Game of Life")


done = False

clock = pygame.time.Clock()


In [31]:
FONT_SIZE = 16

font = pygame.font.Font(None, FONT_SIZE)

In [32]:
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        elif event.type == pygame.MOUSEBUTTONDOWN:
            pos = pygame.mouse.get_pos()
            if ( pos[0] >= WIDTH * COLUMNS + MARGIN * (COLUMNS) 
                or pos[1] >= HEIGHT * ROWS + MARGIN * (ROWS) ):
                continue
            column = pos[0] // (WIDTH + MARGIN)
            row = pos[1] // (HEIGHT + MARGIN)
            grid[row, column, 0] = 1
            print("Click ", pos, "Grid coordinates: ", row, column)

    screen.fill(BLACK)

    for row in range(ROWS):
        for column in range(COLUMNS):
            color = WHITE
            if grid[row, column, 0] == 1:
                color = GREEN
            pygame.draw.rect(screen,
                             color,
                             [(MARGIN + WIDTH) * column + MARGIN,
                              (MARGIN + HEIGHT) * row + MARGIN,
                              WIDTH,
                              HEIGHT])
            
            # render text onto a surface
            text_rabbits = font.render(f"C: {8}", True, (0, 0, 0))
            # blit the surface onto the screen
            screen.blit(text_rabbits, [(MARGIN + WIDTH) * column 
                + MARGIN , (MARGIN + HEIGHT) 
                * row + MARGIN ])

            text_foxes = font.render(f"Z: {5}", True, (0, 0, 0))
            # blit the surface onto the screen
            screen.blit(text_foxes, [(MARGIN + WIDTH) * column 
                + MARGIN , (MARGIN + HEIGHT) 
                * row + MARGIN + FONT_SIZE ])

    # change color of the grid[0, 0] every two seconds
    if pygame.time.get_ticks() % 4000 < 2000:
        grid[0, 0, 0] = 0
    else:
        grid[0, 0, 0] = 1

    clock.tick(60)

    pygame.display.flip()

pygame.quit()

Click  (72, 59) Grid coordinates:  1 2
Click  (203, 123) Grid coordinates:  3 5
Click  (89, 250) Grid coordinates:  7 2
Click  (248, 320) Grid coordinates:  9 7
Click  (286, 184) Grid coordinates:  5 8
Click  (132, 174) Grid coordinates:  4 3
Click  (184, 226) Grid coordinates:  6 5
Click  (136, 329) Grid coordinates:  9 3
Click  (67, 187) Grid coordinates:  5 1
Click  (313, 346) Grid coordinates:  9 8
Click  (334, 340) Grid coordinates:  9 9
Click  (332, 283) Grid coordinates:  8 9
Click  (330, 240) Grid coordinates:  6 9
Click  (330, 212) Grid coordinates:  6 9
Click  (320, 165) Grid coordinates:  4 9
